In [1]:
import os
import pandas as pd

In [2]:
res = pd.read_csv('../results/eurosat_probing.csv', index_col = 0)
res['Data Set'] = 'EuroSat'
res['Dimension'] = res['N Shots'] - 1
all_results = res

datasets = {
            'Flowers102': 'flowers',
            'CIFAR-10': 'cifar10',
            'UCMerced': 'ucmerced',
            'PatternNet': 'patternnet'
            }

base_path = '../results'
suffix = '_probing.csv'
for d_name in datasets.keys():
    d_id = datasets[d_name]
    path = os.path.join(base_path,f'{d_id}{suffix}')
    res = pd.read_csv(path, index_col = 0)
    res['Data Set'] = d_name
    if d_id in ['flowers', 'cifar10']:
        res['Dimension'] = 1
    else:
        res['Dimension'] = res['N Shots'] - 1
    all_results = pd.concat([all_results, res])

In [7]:


# Group by 'N Shots', 'Method', and 'Data Set' and calculate mean and std
summary_df = all_results.groupby(['N Shots', 'Method', 'Data Set', 'Dimension'])['Accuracy'].agg(['mean', 'std']).reset_index()

# Rename columns for clarity
summary_df.rename(columns={'mean': 'Accuracy Mean', 'std': 'Accuracy Std'}, inplace=True)

summary_df = summary_df.sort_values(by = ['Data Set', 'N Shots', 'Method'])

# Display the result
print(summary_df)


    N Shots        Method    Data Set  Dimension  Accuracy Mean  Accuracy Std
0         3      FlagNets    CIFAR-10          1        0.59158      0.007516
5         3     ProtoNets    CIFAR-10          1        0.58432      0.008303
10        3  SubspaceNets    CIFAR-10          1        0.58527      0.008922
15        5      FlagNets    CIFAR-10          1        0.65138      0.008593
20        5     ProtoNets    CIFAR-10          1        0.64460      0.010338
25        5  SubspaceNets    CIFAR-10          1        0.63779      0.009288
30        7      FlagNets    CIFAR-10          1        0.68414      0.007193
35        7     ProtoNets    CIFAR-10          1        0.67886      0.008037
40        7  SubspaceNets    CIFAR-10          1        0.66739      0.007335
1         3      FlagNets     EuroSat          2        0.77443      0.010369
6         3     ProtoNets     EuroSat          2        0.75917      0.008956
11        3  SubspaceNets     EuroSat          2        0.76830 

In [8]:
# Define the custom order for 'Data Set'
data_set_order = ['EuroSat', 'CIFAR-10', 'Flowers102']

# Convert 'Data Set' to a categorical type with the specified order
summary_df['Data Set'] = pd.Categorical(summary_df['Data Set'], categories=data_set_order, ordered=True)


# for n_shots in summary_df['N Shots'].unique():
#     idx = summary_df['N Shots'] == n_shots
pivot_df = summary_df.pivot_table(
    columns= 'Method',
    index= ['N Shots', 'Data Set'],
    values='Accuracy Mean'
)
pivot_df = pivot_df.round(3)
pivot_df = pivot_df.applymap(lambda x: f"{x:.3f}".rstrip('0').rstrip('.') if pd.notnull(x) else '')
print(pivot_df)
print(pivot_df.to_latex(index=True))

Method             FlagNets ProtoNets SubspaceNets
N Shots Data Set                                  
3       EuroSat       0.774     0.759        0.768
        CIFAR-10      0.592     0.584        0.585
        Flowers102    0.894     0.879        0.888
5       EuroSat       0.816     0.798        0.808
        CIFAR-10      0.651     0.645        0.638
        Flowers102    0.923     0.911         0.92
7       EuroSat       0.836     0.817        0.829
        CIFAR-10      0.684     0.679        0.667
        Flowers102    0.935     0.923        0.934
\begin{tabular}{lllll}
\toprule
 & Method & FlagNets & ProtoNets & SubspaceNets \\
N Shots & Data Set &  &  &  \\
\midrule
\multirow[t]{3}{*}{3} & EuroSat & 0.774 & 0.759 & 0.768 \\
 & CIFAR-10 & 0.592 & 0.584 & 0.585 \\
 & Flowers102 & 0.894 & 0.879 & 0.888 \\
\cline{1-5}
\multirow[t]{3}{*}{5} & EuroSat & 0.816 & 0.798 & 0.808 \\
 & CIFAR-10 & 0.651 & 0.645 & 0.638 \\
 & Flowers102 & 0.923 & 0.911 & 0.92 \\
\cline{1-5}
\multirow[t]{3}

/tmp/ipykernel_270979/317577426.py:10: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pivot_df = summary_df.pivot_table(
/tmp/ipykernel_270979/317577426.py:16: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  pivot_df = pivot_df.applymap(lambda x: f"{x:.3f}".rstrip('0').rstrip('.') if pd.notnull(x) else '')
